In [ ]:
import ipywidgets.widgets as widgets
from time import sleep, time
import requests
import json
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
import ballsbot.config as config

config.ENABLE_MULTIPROCESSING = False

In [ ]:
from ballsbot.utils import keep_rps, run_as_thread
from ballsbot.drawing import ManipulatorPoseDrawing, BotPoseSelfCoordsDrawing, StatusMessagesDrawing
from ballsbot.dashboard import Dashboard
from ballsbot.lidar_drawing import LidarDrawing
from ballsbot.lidar import default_calibration

In [ ]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

In [ ]:
def get_controller_state():
    ts = None
    while True:
        ts = keep_rps(ts, fps=1)
        if len(controller.axes):
            break
    
    axes = [it.value for it in controller.axes]
    buttons = [it.value for it in controller.buttons]
    return {
        'axes': axes,
        'buttons': buttons,
    }

In [ ]:
bot_ip = '192.168.0.115'
bot_port = 4443
bot_key = 'FIXME'

In [ ]:
bot_state = None
images_content = [None, None]

def update_bot_state():
    global bot_state, images_content
    
    session = requests.Session()
    
    # POST https://192.168.0.115:4443/auth password
    r = session.post(f'https://{bot_ip}:{bot_port}/auth', data={'key': bot_key}, verify=False)
    r.raise_for_status()
    token = r.json()['token']

    # GET https://192.168.0.115:4443/settings?token=
    r = session.get(f'https://{bot_ip}:{bot_port}/settings', params={'token': token}, verify=False)
    r.raise_for_status()
    settings = r.json()

    ts = None
    while True:
        ts = keep_rps(ts, fps=4)

        # POST https://192.168.0.115:4443/controller_state controller_state token
        r = session.post(f'https://{bot_ip}:{bot_port}/controller_state', data={
            'token': token,
            'controller_state': json.dumps(get_controller_state()),
        }, verify=False)
        r.raise_for_status()
        bot_state = r.json()

        for i in (0, 1):
            # GET https://192.168.0.115:4443/camera_image?token=&image_index=0
            r = session.get(f'https://{bot_ip}:{bot_port}/camera_image', params={'token': token, 'image_index': i}, verify=False)
            r.raise_for_status()
            images_content[i] = r.content

run_as_thread(update_bot_state)

In [ ]:
class LidarStub:
    def get_calibration(self):
        return default_calibration()

    def get_lidar_points(self, **kwargs):
        return bot_state['lidar']

lidar = LidarStub()

images = []
for _ in (0, 1):
    images.append(widgets.Image(format='jpeg', width=320, height=240))
hbox = widgets.HBox(images)
display(hbox)

dashboard = Dashboard(widgets)

lidar_drawing_raw = BotPoseSelfCoordsDrawing(dashboard, 'lidar')
lidar_drawing = LidarDrawing(lidar, lidar_drawing_raw)
dashboard.add_subplot('lidar', lidar_drawing_raw.get_drawing_func_name())

status_drawing_raw = StatusMessagesDrawing(dashboard, 'status')
dashboard.add_subplot('status', status_drawing_raw.get_drawing_func_name())

manipulator_drawing_raw = ManipulatorPoseDrawing(dashboard, 'manipulator_xy', 'manipulator_xz')
dashboard.add_subplot('manipulator_xy', manipulator_drawing_raw.get_drawing_func_name())
dashboard.add_subplot('manipulator_xz', manipulator_drawing_raw.get_drawing_func_name())

display(dashboard.get_image())

In [ ]:
def update_images():
    ts = None
    while True:
        ts = keep_rps(ts, fps=4)
        
        if not images_content[0]:
            continue

        for i in (0, 1):
            images[i].value = images_content[i]

        if not bot_state:
            continue

        lidar_drawing.update_image_once(cached=False)

        capacity = int(round(bot_state['ups'] or -1.))
        distances = bot_state['distance_sensors']
        if distances and 'manipulator' in distances and distances['manipulator']:
            distance = distances['manipulator'].get('distance', -1)
        else:
            distance = -1
        status_drawing_raw.update_image(f'battery charge: {capacity}%\ntarget: {distance} m')

        pose = bot_state['manipulator']['current_pose']
        manipulator_drawing_raw.update_image(pose, override_crop_half_size=750.)

        dashboard.redraw()

run_as_thread(update_images)

In [ ]:
# from ballsbot.utils import join_all_theads
# join_all_theads()